In [9]:
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing 
from sklearn.ensemble import RandomForestClassifier
import torch
import os
from models.dkvmn_text import SUBJ_DKVMN
from data_loaders.assist2009 import ASSIST2009


In [10]:
dataset_label = "correct"
seq_len = 100

# 데이터 수정
df_full = pd.read_csv('./FeatureImportanceDL/dataset/skill_builder_data.csv', encoding='ISO-8859-1').drop(labels=["bottom_hint", "answer_id"], axis=1).dropna()
all_X = df_full.drop(labels=dataset_label, axis=1)

pre_columns = ['type', 'answer_type', 'skill_name', 'tutor_mode', 'answer_text']
encoded_X = all_X.copy()
all_X[pre_columns] = encoded_X[pre_columns].apply(lambda x: preprocessing.LabelEncoder().fit_transform(x))

all_y = df_full[dataset_label]
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
ckpt_path = os.path.join("ckpts", "dkvmn+")
ckpt_path = os.path.join(ckpt_path, "ASSIST2009")



/tmp/ipykernel_1638660/2568725576.py:5: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df_full = pd.read_csv('./FeatureImportanceDL/dataset/skill_builder_data.csv', encoding='ISO-8859-1').drop(labels=["bottom_hint", "answer_id"], axis=1).dropna()


In [13]:
X_tr, X_val, y_tr, y_val = train_test_split(
    all_X,
    all_y,
    test_size = 0.3,
    random_state = 42
)

X_tr, X_te, y_tr, y_te = train_test_split(
    all_X,
    all_y,
    test_size = 0.3,
    random_state = 69
)

dataset = ASSIST2009(seq_len)

model = torch.nn.DataParallel(SUBJ_DKVMN(dataset.num_q, num_qid=dataset.num_pid, 
                                         dim_s=100, size_m=50)).to(device)
model.load_state_dict(torch.load(os.path.join(ckpt_path, "model.ckpt")))

result = permutation_importance(estimator=model, X=X_tr, y=y_tr, scoring="f1", n_repeats=5, random_state=42, n_jobs=2)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


TypeError: estimator should be an estimator implementing 'fit' method, DataParallel(
  (module): SUBJ_DKVMN(
    (k_emb_layer): Embedding(103, 100)
    (d_emb_layer): Embedding(103, 100)
    (transformer_encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
          )
          (linear1): Linear(in_features=100, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=100, bias=True)
          (norm1): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (bertmodel): BertModel(
      (embeddings): BertEmbeddings(
    